# SWOT L2 virtualizarr example
### (NASA CMR Shortname: SWOT_L2_LR_SSH_Basic_2.0)

#### *Author: Ed Armstrong, PO.DAAC*

*Reference herein to any specific commercial product, process, or service by trade name, trademark, manufacturer, or otherwise, does not constitute or imply its endorsement by the United States Government or the Jet Propulsion Laboratory, California Institute of Technology.*

In [1]:
# Built-in packages
import os
import sys

# Filesystem management 
import fsspec
import earthaccess

# Data handling
import xarray as xr
from virtualizarr import open_virtual_dataset

# Parallel computing 
import multiprocessing
import dask
from dask import delayed
import dask.array as da
from dask.distributed import Client

# Pandas
import pandas as pd

# Numpy
import numpy as np

# Other mapping
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Coiled for Dask management
import coiled

In [2]:
 pip list | grep -E  -e  '(^xarray|virtualizarr|earthaccess|kerchunk)'

earthaccess               0.14.0
kerchunk                  0.2.7
virtualizarr              1.3.2
xarray                    2025.1.2
Note: you may need to restart the kernel to use updated packages.


## 1. Get Data File S3 endpoints in Earthdata Cloud 
The first step is to find the S3 endpoints to the files. Handling access credentials to Earthdata and then finding the endpoints can be done a number of ways (e.g. using the `requests`, `s3fs` packages) but we use the `earthaccess` package for its ease of use. We get the endpoints for all files in the CCMP record.

In [3]:
# Get Earthdata creds
earthaccess.login()

Enter your Earthdata Login username:  edward.m.armstrong
Enter your Earthdata password:  ········


In [22]:
# Locate  SWOT L2 file information / metadata for the PGC files:
granule_info_1 = earthaccess.search_data(
    short_name="SWOT_L2_LR_SSH_Basic_2.0",
    granule_name="SWOT_L2_LR_SSH_Basic*PGC*.nc",
    temporal=("2023-07-26", "2024-01-24"),
    )

In [23]:
# Locate  SWOT L2 file information / metadata for the PIC files:
granule_info_2 = earthaccess.search_data(
    short_name="SWOT_L2_LR_SSH_Basic_2.0",
    granule_name="SWOT_L2_LR_SSH_Basic*PIC*.nc",
    temporal=("2024-01-25", "2025-05-03"),
    )

In [24]:
# append the two granule lists. They are sorted in time.
granule_info = granule_info_1 + granule_info_2

In [7]:
# Get AWS creds. Note that if you spend more than 1 hour in the notebook, you may have to re-run this line!!!
fs = earthaccess.get_fsspec_https_session()
#fs = earthaccess.get_s3_filesystem(results=granule_info)

In [25]:
# Get S3 or HTTPS endpoints for all files:
data_s3links = [g.data_links(access="https")[0] for g in granule_info]
#data_s3links = [g.data_links(access="direct")[0] for g in granule_info]
print(len(data_s3links))
print(data_s3links[0:4])

18374
['https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_LR_SSH_2.0/SWOT_L2_LR_SSH_Basic_001_149_20230726T122755_20230726T131924_PGC0_01.nc', 'https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_LR_SSH_2.0/SWOT_L2_LR_SSH_Basic_001_150_20230726T131923_20230726T141051_PGC0_01.nc', 'https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_LR_SSH_2.0/SWOT_L2_LR_SSH_Basic_001_151_20230726T141049_20230726T150135_PGC0_01.nc', 'https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_LR_SSH_2.0/SWOT_L2_LR_SSH_Basic_001_152_20230726T150304_20230726T155344_PGC0_01.nc']


## 2. Generate and open reference files 

In [26]:
# This will be assigned to 'loadable_variables' and needs to be modified per the specific 
# coord names of the data set:
coord_vars = ["num_lines","num_pixels"]
reader_opts = {"storage_options": fs.storage_options,   "timeout": 120} # S3 filesystem creds from previous section.

The reference can be saved to file and used to open the corresponding CCMP data file with Xarray:

In [27]:
# Open data using the reference file, using a small wrapper function around xarray's open_dataset. 
# This will shorten code blocks in other sections. 
def get_vds_mapper(vds_link, in_cloud_region=False):
    """
    Produces a virtudal dataset mapper that can be passed to xarray. 
    
    * vds_link: str, link to the mapper
    * in_cloud_region: bool, True if in cloud in the same region as the data, 
        False otherwise.
    """

    if in_cloud_region:
        fs_data = earthaccess.get_s3_filesystem(daac="PODAAC")
        remote_protocol = "s3"
    else:
        fs_data = earthaccess.get_fsspec_https_session()
        # We set this header to identify usage of the VDS. This helps us support continued development. 
        fs_data.client_kwargs['headers']['User-Agent'] = "virtual-dataset-consumer"
        remote_protocol = "https"
    
    storage_opts = {"fo": vds_link, "remote_protocol": remote_protocol, "remote_options": fs_data.storage_options}
    fs_ref = fsspec.filesystem('reference', **storage_opts)
    return fs_ref.get_mapper('')

### 2.2.1 Process some files using Dask local cluster in parallel
If using an `m6i.4xlarge` AWS EC2 instance, there are 16 CPUs available and each should have enough memory to utilize all at once. If working on a different VM-type, change the `n_workers` in the call to `Client()` below as needed.

In [ ]:
# Check how many cpu's are on this VM:
print("CPU count =", multiprocessing.cpu_count())

In [ ]:
# Start up cluster and print some information about it:
client = Client(n_workers=4, threads_per_worker=1)

dask.config.set({"distributed.worker.connections.outgoing": 5})

print(client.cluster)
print("View any work being done on the cluster here", client.dashboard_link)

In [ ]:
%%time
# Create individual references in memory:
open_vds_par = delayed(open_virtual_dataset)
tasks = [
    open_vds_par(p, indexes={}, reader_options=reader_opts, loadable_variables=coord_vars) 
    for p in data_s3links[:] # All passes
    ]
virtual_ds_list = list(da.compute(*tasks)) # The xr.concat() function below needs a list rather than a tuple.
print(len((virtual_ds_list)))

In [ ]:
%%time

# Create the combined reference. Extract out a start time for each file (pass). Just do a few files.
orbit_start = []
for g in granule_info[:len(data_s3links[:])]:
    datetime_str = g['umm']['TemporalExtent']['RangeDateTime']['BeginningDateTime']
    datetime_obj = np.datetime64(datetime_str)
    orbit_start.append(datetime_obj)

# create panda list for better management and naming
orbit_start = pd.Index(orbit_start, name="orbit")

coords_list = ['latitude', 'longitude' ]
virtual_ds_combined = xr.concat(virtual_ds_list, orbit_start, coords=coords_list, compat='override', combine_attrs='drop_conflicts')

In [ ]:
# Save in JSON and PARQUET format:
fname_combined_json = 'ref_combined_swotl2_full_PIC_3_https.json'
fname_combined_parq = 'ref_combined_swotl2_full__PIC_3_https.parq'
virtual_ds_combined.virtualize.to_kerchunk(fname_combined_json, format='json')
virtual_ds_combined.virtualize.to_kerchunk(fname_combined_parq, format='parquet')

In [ ]:
# shutdown the cluster
client.shutdown()

In [ ]:
%%time

fname_combined_json = '/Users/earmstro/Downloads/SWOT_L2_LR_SSH_Basic_2.0_virtual_https.json'
# Lets open the JSON VDS

vds_link = fname_combined_json
vds_mapper = get_vds_mapper(vds_link, in_cloud_region=False)

## No modification needed!
data = xr.open_dataset(
    vds_mapper, engine="zarr", chunks={}, decode_cf=False, mask_and_scale=True,
    backend_kwargs={"consolidated": False}
)
data


### 2.2.1  Using a Dask Management service (coiled) to parallize and  process the entire record

In [28]:
%%time

## --------------------------------------------
## Create single reference files with parallel computing using Coiled
## --------------------------------------------

# Wrap `open_virtual_dataset()` into coiled function and copy to mulitple VM's:
open_vds_par = coiled.function(
    region="us-west-2", spot_policy="on-demand", 
    vm_type="m6i.large", n_workers=5
    )(open_virtual_dataset)

# Begin computations for the enire record:
results = open_vds_par.map(
    data_s3links[:], indexes={}, 
    reader_options=reader_opts, loadable_variables=coord_vars
    )

virtual_ds_list = []
for r in results:
    virtual_ds_list.append(r)

Output()

╭──────────────────────────────── Package Info ────────────────────────────────╮
│                    ╷                                                         │
│   Package          │ Note                                                    │
│ ╶──────────────────┼───────────────────────────────────────────────────────╴ │
│   coiled_local_bin │ Wheel built from                                        │
│                    │ /scratch/coiled-senv-spec/wheels/coiled_local_bin/coi   │
│                    │ led_local_bin-0.0.0-py3-none-any.whl                    │
│                    ╵                                                         │
╰──────────────────────────────────────────────────────────────────────────────╯

Output()

CPU times: user 1min 1s, sys: 3.83 s, total: 1min 4s
Wall time: 4h 16min 36s


In [29]:
%%time

# Create the combined reference. Extract out a start time for each file (pass). Just do a few files.
orbit_start = []
for g in granule_info[:len(data_s3links[:])]:
    datetime_str = g['umm']['TemporalExtent']['RangeDateTime']['BeginningDateTime']
    datetime_obj = np.datetime64(datetime_str)
    orbit_start.append(datetime_obj)

# Create panda list for better management and naming
# Sort the orbits so they algin monotonically in time
orbit_start = pd.Index(orbit_start, name="orbit").sort_values()

coords_list = ['latitude', 'longitude' ]
virtual_ds_combined = xr.concat(virtual_ds_list, orbit_start, coords=coords_list, compat='override', combine_attrs='drop_conflicts')

<timed exec>:5: UserWarning: no explicit representation of timezones available for np.datetime64


CPU times: user 1min 17s, sys: 292 ms, total: 1min 17s
Wall time: 1min 17s


In [30]:
# Save in JSON and PARQUET format:
fname_combined_json = 'ref_combined_swotl2_https_sorted.json'
fname_combined_parq = 'ref_combined_swotl2_https_sorted.parq'
virtual_ds_combined.virtualize.to_kerchunk(fname_combined_json, format='json')
virtual_ds_combined.virtualize.to_kerchunk(fname_combined_parq, format='parquet')

In [18]:
# shutdown cluster
open_vds_par.cluster.shutdown()

In [ ]:
%%time


fname_combined_json = 'ref_combined_swotl2_https_sorted.json'

# Lets open the JSON VDS
vds_link = fname_combined_json

# Lets open the JSON VDS
vds_mapper = get_vds_mapper(vds_link, in_cloud_region=False)

## No modification needed!
data = xr.open_dataset(
    vds_mapper, engine="zarr", chunks={},
    backend_kwargs={"consolidated": False}
)
data


In [ ]:


# Load just the orbit coordinate (tiny)
orbit_time = data.orbit.load()

fig, ax = plt.subplots(figsize=(10, 4))

ax.plot(orbit_time.values, range(len(orbit_time)), '.', markersize=1)

ax.set_xlabel("Time")
ax.set_ylabel("Orbit index")
ax.set_title("SWOT Orbit Times")

plt.tight_layout()
plt.show()


In [ ]:
# Make some simple plots over a few orbits to visualize the data

#orbits = [0, 1, 2, 3, 12, 15]
orbits = [3,12]


fig, axes = plt.subplots(
    1, 2, figsize=(14, 14),
    subplot_kw={"projection": ccrs.PlateCarree()}
)
axes = axes.ravel()

# mask the missing values bigger than 2147483
for ax, k in zip(axes, orbits):

    ssh = data.ssh_karin.isel(orbit=k)
    #ssh = ssh.where(ssh < 2147483).load()
    ssh = ssh.where(ssh < 1000).load()
    lat = data.latitude.isel(orbit=k).load()
    lon = data.longitude.isel(orbit=k).load()

    #print("Max after masking:", float(ssh.max()))
    #print(np.nanmax(ssh.values))
    #print("NaNs:", np.isnan(ssh.values).sum())

    
    # light decimation (huge speedup, no visual loss)
    ssh = ssh[:, ::3]
    lat = lat[:, ::3]
    lon = lon[:, ::3]

    pcm = ax.pcolormesh(
        lon,
        lat,
        ssh,
        transform=ccrs.PlateCarree(),
        shading="auto",
        cmap="viridis",
    )

    ax.coastlines()
    ax.set_title(f"Orbit {k}\n{str(data.orbit.values[k])[:19]}")
    
# --- after the plotting loop ---

# leave room at the bottom
fig.subplots_adjust(bottom=0.10)

# create a colorbar axis spanning the figure width
cax = fig.add_axes([0.15, 0.05, 0.7, 0.025])  # [left, bottom, width, height]

fig.colorbar(
    pcm,
    cax=cax,
    orientation="horizontal",
    label="ssh_karin (m)"
)

plt.show()



In [ ]:


k = 3   # orbit index you want

# Load this orbit only
ssh = data.ssh_karin.isel(orbit=k)
lat = data.latitude.isel(orbit=k)
lon = data.longitude.isel(orbit=k)

# Mask bad fill values
ssh = ssh.where(ssh < 1000)

# Define Indian Ocean box
lat_min, lat_max = -60, 30
lon_min, lon_max = 20, 120

region_mask = (
    (lat >= lat_min) & (lat <= lat_max) &
    (lon >= lon_min) & (lon <= lon_max)
)

ssh_io = ssh.where(region_mask)
lat_io = lat
lon_io = lon

# Load into memory (only masked region gets computed)
ssh_io = ssh_io.load()
lat_io = lat_io.load()
lon_io = lon_io.load()



In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.coastlines()

pcm = ax.pcolormesh(
    lon_io,
    lat_io,
    ssh_io,
    shading="auto",
    cmap="viridis",
    transform=ccrs.PlateCarree()
)

plt.colorbar(pcm, ax=ax, label="ssh_karin (m)")
plt.title(f"Orbit {k} – Indian Ocean")
plt.show()


In [ ]:
# Load just the orbit coordinate (tiny)
orbit_time = data.orbit.load()

fig, ax = plt.subplots(figsize=(10, 4))

ax.plot(orbit_time.values, range(len(orbit_time)), '.', markersize=1)

ax.set_xlabel("Time")
ax.set_ylabel("Orbit index")
ax.set_title("SWOT Orbit Times")

plt.tight_layout()
plt.show()


In [ ]:
data_jan = data.sel(orbit=slice("2023-08-01", "2023-08-01"))
print(data_jan.sizes["orbit"])

lat_min, lat_max = -50, 30
lon_min, lon_max = 20, 120


In [ ]:
lat = data_jan.latitude
lon = data_jan.longitude

region_mask = (
    (lat >= lat_min) & (lat <= lat_max) &
    (lon >= lon_min) & (lon <= lon_max)
)

ssh = data_jan.ssh_karin

# mask fill values first
ssh = ssh.where(ssh < 1000)

# apply spatial mask
ssh_region = ssh.where(region_mask)


In [ ]:
valid_orbits = []

for k in range(data_jan.sizes["orbit"]):

    lat_k = data_jan.latitude.isel(orbit=k)
    lon_k = data_jan.longitude.isel(orbit=k)

    intersects = (
        ((lat_k >= lat_min) & (lat_k <= lat_max) &
         (lon_k >= lon_min) & (lon_k <= lon_max))
    ).any()

    if intersects:
        valid_orbits.append(k)

print("Orbits intersecting region:", valid_orbits)
print("Count:", len(valid_orbits))


In [ ]:


fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent([lon_min, lon_max, lat_min, lat_max])
ax.coastlines()

#for k in range(ssh_region.sizes["orbit"]):
for k in valid_orbits:
    ssh_i = ssh_region.isel(orbit=k).load()
    lat_i = lat.isel(orbit=k).load()
    lon_i = lon.isel(orbit=k).load()
    
    pcm = ax.pcolormesh(
        lon_i,
        lat_i,
        ssh_i,
        shading="auto",
        cmap="viridis",
        vmin=-40,
        vmax=40,
        transform=ccrs.PlateCarree()
    )

plt.colorbar(pcm, ax=ax, label="ssh_karin (m)")
plt.title("SWOT SSH — Orbits for January 2024 — Indian Ocean")
plt.show()
